In [11]:
import boto3

In [14]:
!pwd

/home/ec2-user/SageMaker/mlops_aws_sagemaker/docker


In [15]:
%%writefile /home/ec2-user/SageMaker/mlops_aws_sagemaker/Dockerfile

FROM python:3.9-slim-buster

COPY requirements.txt /opt/ml/requirements.txt
COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

RUN pip3 install -r /opt/ml/requirements.txt

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3", "/opt/ml/code/train.py"]

Writing /home/ec2-user/SageMaker/mlops_aws_sagemaker/Dockerfile


In [16]:
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
ecr_repository = "prophet_training"
tag = ":latest"
processing_repository_uri = "{}.dkr.ecr.{}.amazonaws.com/{}".format(
    account_id, region, ecr_repository + tag
)

In [20]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository /home/ec2-user/SageMaker/mlops_aws_sagemaker/

Sending build context to Docker daemon  5.346MB
Step 1/9 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/9 : COPY requirements.txt /opt/ml/requirements.txt
 ---> eb3f63e3c306
Step 3/9 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> 75ea701f2699
Step 4/9 : COPY code/train.py /opt/ml/code/train.py
 ---> 311b250d4901
Step 5/9 : RUN pip3 install -r /opt/ml/requirements.txt
 ---> Running in efe49776b65c
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━

In [24]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [25]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'prophet_training' already exists in the registry with id '654654222480'


In [26]:
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
# if [ $? -ne 0 ]
# then
# aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

In [27]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [28]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.ap-south-1.amazonaws.com/prophet_training]

65c74f63: Preparing 
21590e13: Preparing 
e1dcf4a8: Preparing 
c4d8e503: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
65c74f63: Pushed   657.9MB/627.5MBAlatest: digest: sha256:55d8c0561ac2b11bbfdcff3c1f97acb48f534f8ab1cc2de5669de5fbf72473df size: 2204
